<a href="https://colab.research.google.com/github/roberthsu2003/Transformer/blob/main/%E5%AF%A6%E6%88%B0%E9%81%8B%E7%94%A8/NER/ner1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install transformers datasets evaluate seqeval

1. 載入相關套件

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
import evaluate
from datasets import load_dataset

2使用數據集

In [ ]:
ner_datasets = load_dataset("peoples_daily_ner",cache_dir='./data',trust_remote_code=True)

In [ ]:
#查看第一筆資料
ner_datasets['train'][0]

In [ ]:
#查看ner_tags,5,6,所代表的意思
ner_datasets['train'].features

In [ ]:
#取得label_list
label_list = ner_datasets['train'].features['ner_tags'].feature.names
label_list

3數據預處理

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-chinese")

In [ ]:
#不正確,這是以字為單位,而不是以句子為單位
tokenizer(ner_datasets['train'][0]["tokens"])

In [ ]:
tokenizer(ner_datasets['train'][0]['tokens'], is_split_into_words=True) #必需使用is_split_into_words

In [ ]:
#英文的拆字,注意數量(10673, 12865, 12921, 8181, 8681)
res = tokenizer('interesting word')
res

In [ ]:
# 0,0,0,0代表第1個字
# 1代表第2個字
res.word_ids()

In [ ]:
from pprint import pprint
def process_batch(examples):
    tokenized_examples = tokenizer(examples['tokens'], max_length=128, truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_examples.word_ids(batch_index=i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenized_examples['labels'] = labels

    return tokenized_examples


In [ ]:
tokenized_datasets = ner_datasets.map(process_batch,batched=True)
tokenized_datasets['train'][0]

In [ ]:
from pprint import pprint
pprint(tokenized_datasets['train'][0])

4建立模型

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("google-bert/bert-base-chinese",num_labels=len(label_list)) #預設是2,現在是7個

In [ ]:
model.num_labels

In [ ]:
seqeval = evaluate.load('seqeval')
seqeval

In [ ]:
import numpy as np

def eval_metric(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=-1)

    true_predictions =[
        [label_list[p] for p,l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [label_list[l] for p, l in zip(prediction, label) if l != -100]
       for prediction, label in zip(predictions, labels)
    ]

    result = seqeval.compute(predictions=true_predictions, references=true_labels, mode="strict", scheme="IOB2")
    return {
        "f1": result["overall_f1"]
    }

配置訓練參數

In [ ]:
args = TrainingArguments(
    output_dir = "models_for_ner",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    eval_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    logging_steps = 50,
    num_train_epochs=3,
    report_to='none'
)

建立訓練器

In [ ]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=eval_metric,
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
)

In [ ]:
trainer.train()

評估模型

In [ ]:
trainer.evaluate(eval_dataset=tokenized_datasets['test'])

上傳模型

In [ ]:
from huggingface_hub import login
login()

In [ ]:
trainer.push_to_hub("roberthsu2003") #由於有設./checkpoints,所以自動產生checkpoints的repo,也會自動上傳評估至repo
#同時要上傳tokenizer
model_name = "roberthsu2003/models_for_ner"
tokenizer.push_to_hub(model_name)


下載模型
模型預測
pipeline()->使用NER的官方說明
https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.TokenClassificationPipeline

In [ ]:
from transformers import pipeline,AutoModelForTokenClassification,AutoTokenizer

model = AutoModelForTokenClassification.from_pretrained('roberthsu2003/models_for_ner')
tokenizer = AutoTokenizer.from_pretrained('roberthsu2003/models_for_ner')
ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
ner_pipe("徐國堂在台北上班")

In [ ]:
#顯示
print(model.config.id2label)

In [ ]:
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
model.config.id2label = {idx: label for idx, label in enumerate(label_list)}
model.config.id2label

In [ ]:
from huggingface_hub import login
login()


In [ ]:
model = AutoModelForTokenClassification.from_pretrained('roberthsu2003/models_for_ner')

tokenizer = AutoTokenizer.from_pretrained('roberthsu2003/models_for_ner')
model_name = "roberthsu2003/models_for_ner"
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
model.config.id2label = {idx: label for idx, label in enumerate(label_list)}
model.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)

In [ ]:
ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)
ner_pipe("徐國堂在台北上班")

In [ ]:
model = AutoModelForTokenClassification.from_pretrained('roberthsu2003/models_for_ner')
tokenizer = AutoTokenizer.from_pretrained('roberthsu2003/models_for_ner')
ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)
inputs = "徐國堂在台北上班" 
res = ner_pipe(inputs)

ner_result = {}
for r in res:
  if r["entity_group"] not in ner_datasets:
    ner_result[r['entity_group']] = []
  ner_result[r['entity_group']].append(inputs[r['start']:r['end']])
ner_result

In [ ]:
#模型使用方式,pipline的使用方式
ner_pipe = pipeline("token-classification", model='roberthsu2003/models_for_ner',aggregation_strategy="simple")
inputs = "徐國堂在台北上班" 
res = ner_pipe(inputs)

ner_result = {}
for r in res:
  if r["entity_group"] not in ner_datasets:
    ner_result[r['entity_group']] = []
  ner_result[r['entity_group']].append(inputs[r['start']:r['end']])
ner_result

In [ ]:
#使用model,tokenizer的使用方法
from transformers import AutoModelForTokenClassification, AutoTokenizer
import numpy as np

# Load the pre-trained model and tokenizer
model = AutoModelForTokenClassification.from_pretrained('roberthsu2003/models_for_ner')
tokenizer = AutoTokenizer.from_pretrained('roberthsu2003/models_for_ner')

# The label mapping (you might need to adjust this based on your training)
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

def predict_ner(text):
    """Predicts NER tags for a given text using the loaded model."""
    # Encode the text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    
    # Get model predictions
    outputs = model(**inputs)
    predictions = np.argmax(outputs.logits.detach().numpy(), axis=-1)
    
    # Get the word IDs from the encoded inputs
    # This is the key change - word_ids() is a method on the encoding result, not the tokenizer itself
    word_ids = inputs.word_ids(batch_index=0)
    
    pred_tags = []
    for word_id, pred in zip(word_ids, predictions[0]):
        if word_id is None:
            continue  # Skip special tokens
        pred_tags.append(label_list[pred])

    return pred_tags

#To get the entities, you'll need to group consecutive non-O tags:

def get_entities(tags):
    """Groups consecutive NER tags to extract entities."""
    entities = []
    start_index = -1
    current_entity_type = None
    for i, tag in enumerate(tags):
        if tag != 'O':
            if start_index == -1:
                start_index = i
                current_entity_type = tag[2:] # Extract entity type (e.g., PER, LOC, ORG)
        else: #tag == 'O'
            if start_index != -1:
                entities.append((start_index, i, current_entity_type))
                start_index = -1
                current_entity_type = None
    if start_index != -1:
        entities.append((start_index, len(tags), current_entity_type))
    return entities

# Example usage:
text = "徐國堂在台北上班"
ner_tags = predict_ner(text)
print(f"Text: {text}")
#==output==
#Text: 徐國堂在台北上班
print(f"NER Tags: {ner_tags}")
#===output==
#NER Tags: ['B-PER', 'I-PER', 'I-PER', 'O', 'B-LOC', 'I-LOC', 'O', 'O']


entities = get_entities(ner_tags)
word_tokens = tokenizer.tokenize(text)  # Tokenize to get individual words
print(f"Entities:")
for start, end, entity_type in entities:
    entity_text = "".join(word_tokens[start:end])
    print(f"- {entity_text}: {entity_type}")

#==output==
#Entities:
#- 徐國堂: PER
#- 台北: LOC
